# Multi-Qubit Noisy Simulator
*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial will introduce how to use multi-qubit simulator at the pulse level. The outline is as follows:
- Introduction
- Preparation
- Use multi-qubit noisy simulator at the gate level
- Use multi-qubit noisy simulator at the pulse level
    - Modeling the system 
    - Rabi oscillation
    - Cross-Resonance effect
    - ZZ crosstalk characterization through Ramsey experiment
- Summary

## Introduction

Simulating time evolution of the qubits at the pulse level gives us more insight into the physics of quantum gates and the effects of noise. For superconducting quantum circuits, the transmon qubits are controlled by applying microwave pulses and magnetic flux. However, the performance of quantum gates is often suppressed by various factors: the decoherence of the qubit due to its interaction with the environment, the unwanted crosstalk effect, and leakage into the higher levels of the transmon. 

The multi-qubit noisy simulator provided by Quanlse allows us to simulate quantum operations on a noisy quantum device consisting of multiple transmon qubits to understand the physics behind quantum computing better. Several main types of noise are included in our noisy simulator: decoherence noise, amplitude noise, and crosstalk noise. We will focus on several common applications in superconducting quantum computing based on this noisy simulator: Rabi oscillation, Cross-Resonance effect, and characterizing ZZ crosstalk through a Ramsey experiment.

## Preparation

After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
from Quanlse.remoteOptimizer import remoteOptimize1Qubit as optimize1q

from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

from Quanlse.Simulator import PulseModel
from Quanlse.Simulator.PulseSim3Q import pulseSim3Q

from Quanlse.QWaveform import QJob, QJobList
from Quanlse.QOperator import driveX, driveY, a, sigmaZ, number, driveZ
from Quanlse.QWaveform import square, gaussian
from Quanlse.Utils.Functions import basis, tensor, expect, dagger, partialTrace, project, computationalBasisList, population
from Quanlse.Utils.Bloch import rho2Coordinate, plotBloch
from Quanlse.Utils.Plot import plotBarGraph
from Quanlse.QOperation.FixedGate import H, X, Y, CR, Z, CZ
from Quanlse.QOperation.RotationGate import RZ
from Quanlse.Scheduler.Superconduct import SchedulerSuperconduct
from Quanlse.Utils.Infidelity import unitaryInfidelity

from math import pi
from scipy.optimize import curve_fit

import numpy as np
import matplotlib.pyplot as plt

To use Quanlse Cloud Service, we need to acquire a token to get access to the cloud. 

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ""

## Use multi-qubit noisy simulator at the gate level

We can also run the simulator at the gate level. In this section, we use a predefined `PulseModel()` instance with the default configuration. To create a 3-qubit physics model, we first instantiate the `PulseModel()` object by calling `pulseSim3Q()`.

In [ ]:
model = pulseSim3Q()

To define a circuit of creating GHZ state by Quanlse scheduler, we add the gates to the model by `gate(model.Q[index])`. 

In [ ]:
# Hadamard gate 
H(model.Q[0])

# CNOT: 0 -> 1
H(model.Q[1])
CZ(model.Q[0], model.Q[1])
H(model.Q[1])

# CNOT: 1 -> 2
H(model.Q[2])
CZ(model.Q[1], model.Q[2])
H(model.Q[2])

The pulse sequence of the quantum circuit defined above is generated by calling method `model.schedule`. 

In [ ]:
scheJob = model.schedule()

Define the initial state $|000\rangle$ and run the simulation. Then, plot the probability distribution of different outcome.

In [ ]:
# Run the simulation
psi0 = tensor(basis(3, 0), basis(3, 0), basis(3, 0))
res = runHamiltonian(ham=model.ham, state0=psi0, job=scheJob)

In [ ]:
# Plot the result
popList = [abs(item ** 2) for item in res[0]['state'].T[0]]

basisList = computationalBasisList(3, 3)

plotBarGraph(basisList, popList, "Result", "Outcome", "Population")

As it shows, the measurement result included unexpected values due to the noise. The simulation under decoherence can be done by setting the parameter `isOpen=True` in module `runHamiltonian()`, which takes some time to run, and we will obtain the density matrix after the simulation. For more details about how decoherence noise affects superconducting quantum computing, please refer to [Single-Qubit Noisy Simulator](https://quanlse.baidu.com/#/doc/tutorial-single-qubit-noisy-simulator).

## Use multi-qubit noisy simulator at the pulse level

Multi-qubit noisy simulator supports the quantum control simulation at the pulse level - get the system's final state by defining the waveform of the pulse and other related parameters. It allows us to simulate the quantum control of the superconducting hardware at the lower level. Here, we simulate some of the common operations used in real experiments.  

### Modeling the system

Usually, we use Duffing oscillator to describe the physics model of the superconducting circuits. In lab frame, the system Hamiltonian of a three-qubit system with coupling between qubits $q_0$ and $q_1$, $q_1$ and $q_2$ reads:

$$
\hat{H} = \sum_{i=0}^2 \omega_i \hat{a}^\dagger_i \hat{a}_i + \sum_{i=0}^2 \frac{\alpha_i}{2} \hat{a}^\dagger_i \hat{a}^\dagger_i \hat{a}_i \hat{a}_i + g_{01} (\hat{a}^\dagger_0 \hat{a}_1 + \hat{a}_0 \hat{a}^\dagger_1) +  g_{12} (\hat{a}^\dagger_1 \hat{a}_2 + \hat{a}_1 \hat{a}^\dagger_2),
$$

where $\omega_i$ and $\alpha_i$ are the qubit frequency and anharmonicity of qubit $q_i$ respectively; $g_{i, j}$ is the coupling strength between qubit $q_i$ and qubit $q_j$; $a_i$, $a^\dagger_i$ denote the annihilation operator and creation operator of qubit $q_i$.

In this tutorial, we use a three-qubit system as an example. We first define the parameters of the hardware.

In [ ]:
qubitNum = 3  # The number of qubits
level = 3  # The energy level for each qubit

anharm = -0.33 * 2 * pi  # The anharmonicity of the qubit, in 2 pi GHz
wq0 = 4.914 * 2 * pi  # The frequency for qubit 0, in 2 pi GHz 
wq1 = 5.100 * 2 * pi  # The frequency for qubit 1, in 2 pi GHz
wq2 = 5.200 * 2 * pi  # The frequency for qubit 2, in 2 pi GHz
g01 = 0.0038 * 2 * pi  # The coupling strength of the interaction between qubit 0 and qubit 1, in 2 pi GHz
g12 = 0.0020 * 2 * pi  # The coupling strength of the interaction between qubit 1 and qubit 2, in 2 pi GHz

dt = 1.  # The sampling time of AWG

# T1 relaxation time for qubit 0, qubit 1, and qubit 2, in nanoseconds
t01 = 1000  
t11 = 1120
t21 = 1300

# T2 dephasing time for qubit 0, qubit 1, and qubit 2, in nanoseconds
t02 = 500
t12 = 450
t22 = 600

# The random amplitude distortion
ampNoise = 0.02

The physics model is created by instantiating an object of class `PulseModel`. The types of noise include $T_1$-relaxation noise, $T_2$-dephasing, and distortion of amplitudes.  

In [ ]:
qubitFreq = {0: wq0, 1: wq1, 2: wq2}  # Qubit frequency for each qubit
qubitAnharm = {0: anharm, 1: anharm, 2: anharm}  # Qubit anharmonicity for each qubit
qubitT1 = {0: t01, 1: t11, 2: t21}  # Relaxation time 
qubitT2 = {0: t02, 1: t12, 2: t22}  # Dephasing time
couplingMap = {(0, 1): g01, (1, 2): g12}  # Coupling map

# Create an instant of PulseModel
model = PulseModel(subSysNum=qubitNum,
                   sysLevel=level,
                   qubitFreq=qubitFreq,
                   qubitAnharm=qubitAnharm,
                   couplingMap=couplingMap,
                   T1=qubitT1,
                   T2=qubitT2,
                   dt=dt,
                   ampSigma=ampNoise)

We have constructed a noisy simulator including three superconducting qubits with three types of noises. The next step is to create a `QHamiltonian` object by calling method `createQHamiltonian()`. 

In [ ]:
ham = model.createQHamiltonian()

### Cross-Resonance effect 

The all-microwave control is one of the strategies to realize quantum control on superconducting circuits. In this strategy, two-qubit operations harness the cross-resonance effect of two weakly-coupled qubits. This is done by driving the control qubit with the frequency of the weakly-coupled target qubit. Ideally, the desired $\hat{\sigma}_z \otimes \hat{\sigma}_x$ interaction between the control and target qubit is dominating interaction \[1\]. For more details about CR gate, please refer to [Cross-Resonance Gate](https://quanlse.baidu.com/#/doc/tutorial-cr).

In our simulation, we again drive qubit $q_0$ (control qubit) by various amplitudes (with the drive frequency of the qubit $q_1$). This can be done by `addWaveRot(index, waves, detuning)` where `index` is the index of qubit acted upon; `waves` is the waveform of the pulse; and `detuning` $\Delta$ is the frequency difference ($\Delta = \omega_q - \omega_d$, where $\omega_q$ is the qubit frequency and $\omega_d$ the drive frequency). 

Here, we vary the amplitudes of the pulse and record the population of $|1\rangle$ for each qubit. In this example, $q_1$ is the control qubit driven by the pulse with the frequency of target qubit $q_0$.

In [ ]:
dCoef = 0.03 * (2 * pi)  # The drive strength of the pulse
ampCR = np.linspace(0, 0.5, 40)  # The amplitudes in arbitrary unit 
amps = ampCR * dCoef  
detuning = wq1 - wq0  # The detuning of the pulse

# jobList = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt, title='cr')
jobList = ham.createJobList()

# Fix the gate time
tg = 950

# Append each job to the jobList
for amp in amps:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job = ham.createJob()
    job.addWaveRot(1, waves=square(0, tg, amp), detuning=detuning)  # Apply pulse at qubit 1
    job = model.getSimJob(job)
    jobList.addJob(jobs=job)

Run the simulation with initial state $|\psi\rangle = |010\rangle$, and the control qubit $q_1$ starts at the excited state.

In [ ]:
# Define the initial state of |010>
psi0 = tensor(basis(level, 0), basis(level, 1), basis(level, 0))  

# Run the simulation
result = runHamiltonian(ham=ham, state0=psi0, jobList=jobList)

Define the projector of the first excited state for qubit $q_i$ and initialize the list of expected values.

In [ ]:
prj01 = tensor(basis(3, 1) @ dagger(basis(3,1)), np.identity(level), np.identity(level))  # The projector of qubit 0
prj11 = tensor(np.identity(level), basis(3, 1) @ dagger(basis(3,1)), np.identity(level))  # The projector of qubit 1
prj21 = tensor(np.identity(level), np.identity(level), basis(3, 1) @ dagger(basis(3,1)))  # The projector of qubit 1

Compute the expected values of the projector of each qubit, and plot them with respect to the different amplitudes.

In [ ]:
# Initialize the list of expected values
num0List = []
num1List = []
num2List = []

for res in result.result:
    state = res['state']  # The final state of each job
    num0Expect = expect(prj01, state)  # Compute the expected values of the projector |1><1|
    num1Expect = expect(prj11, state)
    num2Expect = expect(prj21, state)
    num0List.append(num0Expect)
    num1List.append(num1Expect)
    num2List.append(num2Expect)

plt.figure(figsize=(8, 6))
# Plot the figure of CR effect
plt.plot(ampCR, num0List, label='qubit0')
plt.plot(ampCR, num1List, label='qubit1')
plt.plot(ampCR, num2List, label='qubit2')

plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title('Cross-Resonance effect')
plt.legend()
plt.show()

As it shows, the projector $|1\rangle \langle 1|$'s expected value of target qubit $q_0$ changes over the amplitude, while the control qubit $q_1$ is in the excited state when the amplitude is relatively small. Meanwhile, qubit $q_2$ is always in the ground state. It can also be seen that the increasing amplitude inevitably affects qubit $q_1$.

### ZZ crosstalk characterization by a Ramsey experiment

ZZ crosstalk is the major source of unwanted interaction between coupled qubits. It arises from the existence of states of higher energy levels. The effective Hamiltonian of two coupled qubits (directly or indirectly) in the two-qubit subspace is \[2\]:

$$
\hat{H}_{\rm eff} = \omega_{0}\frac{\hat{\sigma}_{z}^0 \otimes I_1}{2} + \omega_{1}\frac{I_0\otimes\hat{\sigma}_{z}^1}{2} + \xi \frac{\hat{\sigma}_{z}^0 \otimes \hat{\sigma}_{z}^1}{2},
$$

Where $\omega_0$, $\omega_1$ are the qubit frequencies and $\xi$ is the strength of ZZ crosstalk. $\xi$ is defined as the different of transition frequencies between $|11\rangle \leftrightarrow |10\rangle$ and $|01\rangle \leftrightarrow |00\rangle$:

$$
\xi = \left(E_{11} - E_{10}\right) - \left(E_{01} - E_{00}\right),
$$

where $E_{ij}$ is the energy level of state $|ij\rangle$. We can actually detect and measure this frequency shift-induced crosstalk by Ramsey experiment. This can be done by applying two Hadamard gates with an idle time apart \[3\]. 

To better illustrate the effect of ZZ crosstalk, we define a new 3-qubit model with stronger coupling strengths (6 ~ 40 MHz).

In [ ]:
dt = 0.2  # The sampling time
level = 3  # The system level
qubitNum = 3  # The number of qubits

g01 = 0.0377 * (2 * pi)
g12 = 0.0060 * (2 * pi)

# Coupling map
couplingMap = {
    (0, 1): g01,
    (1, 2): g12
}

# Qubits frequency anharmonicity
anharm = - 0.33 * (2 * pi)
qubitAnharm = {0: anharm, 1: anharm, 2: anharm}  # The anharmonicities for each qubit

# Qubit Frequency
qubitFreq = {
            0: 5.5904 * (2 * pi),
            1: 4.7354 * (2 * pi),
            2: 4.8524 * (2 * pi)
            }

Create the physics model by class `PulseModel()`, and create Hamiltonian `ham` by the model.

In [ ]:
model = PulseModel(subSysNum=qubitNum, sysLevel=level, couplingMap=couplingMap,
                    qubitFreq=qubitFreq, dt=dt, qubitAnharm=qubitAnharm)
    
ham = model.createQHamiltonian()

Generate the pulses of the gates $H$ and $X$ on different qubits.

In [ ]:
# Define function to generate the QJob for gate of specified qubit 
def generateGate(gate, index):
    job1q, _ = optimize1q(ham=ham.subSystem(index), uGoal=gate.getMatrix(), targetInfid=1e-5)
    job3q = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    waves = job1q.waves
    ops = job1q.ctrlOperators
    for key, op in ops.items():
        job3q.addWave(operators=op, onSubSys=index, waves=waves[key])
     
    return job3q

# Generate the gates needed
h0 = generateGate(H, 0)  # H gate on qubit 0 
h1 = generateGate(H, 1)  # H gate on qubit 1
x1 = generateGate(X, 1)  # X gate on qubit 1
x2 = generateGate(X, 2)  # X gate on qubit 2

In [ ]:
maxTime = 500  # The delayed time in Ramsey experiment, in nanosecond.
freq = 3 / maxTime  # Detuning. 

# Generate job for delayed time 
def generateIdle(tg, index):
    jobIdle = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    jobIdle.addWave(operators=driveZ(level), onSubSys=index, waves=square(0, tg, 2 * pi * freq))
    return jobIdle

Define two different `jobList` objects - one begins with $|00\rangle$ and the other $|01\rangle$ by applying a $X$ gate on qubit $q_1$. Then perform Ramsey experiment on qubit $q_0$. 

In [ ]:
# jobList with initial state |00>
jobListGrd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# jobList with initial state |01> (by applying X gate) 
jobListExd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# Define the delayed time
tgList = np.linspace(0, maxTime, 50)

# Define jobList with initial state |00>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += h0
    job += generateIdle(tg, 0)
    job += h0
    jobListGrd.addJob(job)

# Define jobList with initial state |01>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += x1
    job += h0
    job += generateIdle(tg, 0)
    job += h0
    jobListExd.addJob(job)

# Run the simulation
stateInit = tensor(basis(level, 0), basis(level, 0), basis(level, 0))
resultGrd = runHamiltonian(ham, state0=stateInit, jobList=jobListGrd)
resultExd = runHamiltonian(ham, state0=stateInit, jobList=jobListExd)

Plot the population of excited state of qubit $q_0$ versus delayed time. 

In [ ]:
num0List = []
num1List = []

# projector |1><1| of qubit 0
prj1 = tensor(basis(level, 1) @ dagger(basis(level, 1)), np.identity(9))

# append the result to the list
for res0, res1 in zip(resultGrd, resultExd):
    psi0, psi1 = res0['state'], res1['state']
    rho0, rho1 = psi0 @ dagger(psi0), psi1 @ dagger(psi1)
    num0List.append(expect(prj1, rho0))
    num1List.append(expect(prj1, rho1))

# plot the result
plt.figure(figsize=(8, 6))
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 0')
plt.xlabel
plt.legend()
plt.show()

The strength of ZZ crosstalk $\xi$ can be estimated by computing the frequency difference in the Ramsey oscillation. Therefore, we use the cosine function to fit the result acquired by simulation to compute the frequencies $f_1$, $f_2$. The strength is given by $\xi / \left( 2\pi \right) = |f_1 - f_2|$.

In [ ]:
# Define the fitting curve
def fit(x, omega, theta):
    return - 0.5 * np.cos(omega * x + theta) + 0.5

# Fit the curve
para1Fit, _ = curve_fit(fit, tgList, num0List, [2.1 * pi * freq, 0])
para2Fit, _ = curve_fit(fit, tgList, num1List, [2 * pi * freq, 0])
step = 0.01
y1Fit = [fit(x, para1Fit[0], para1Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]
y2Fit = [fit(x, para2Fit[0], para2Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]

# Plot the curve
plt.figure(figsize=(8, 6))
plt.plot(np.arange(tgList[0], tgList[-1], step), y1Fit)
plt.plot(np.arange(tgList[0], tgList[-1], step), y2Fit)
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 0')
plt.title('Ramsey on Q0')
plt.legend()
plt.show()

In [ ]:
# Calculate the crosstalk strength
xiEst = abs(para1Fit[0] - para2Fit[0]) 
print(f'Coupling strength: {g01 * 1e3 / (2 * pi)} MHz')
print(f'ZZ crosstalk strength: {xiEst * 1e3 / (2 * pi)} MHz')

Due to the strong coupling strength between qubits $𝑞_0$ and $𝑞_1$, it can be observed that the frequency difference is relatively large, that is, the 𝑍𝑍 crosstalk is relatively large.

We can repeat the same experiment to calculate ZZ crosstalk strength $\xi$ between qubit $q_1$ and qubit $q_2$ with smaller coupling strength.

In [ ]:
# jobList with initial state |00>
jobListGrd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# jobList with initial state |01> (by applying X gate)
jobListExd = QJobList(subSysNum=qubitNum, sysLevel=level, dt=dt)

# Define the delayed time
tgList = np.linspace(0, maxTime, 50)

# Define jobList with initial state |00>
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += h1
    job += generateIdle(tg, 1)
    job += h1
    jobListGrd.addJob(job)

# Define jobList with initial state |01>    
for tg in tgList:
    job = QJob(subSysNum=qubitNum, sysLevel=level, dt=dt)
    job += x2
    job += h1
    job += generateIdle(tg, 1)
    job += h1
    jobListExd.addJob(job)

# Run the simulation    
stateInit = tensor(basis(level, 0), basis(level, 0), basis(level, 0))
resultGrd = runHamiltonian(ham, state0=stateInit, jobList=jobListGrd)
resultExd = runHamiltonian(ham, state0=stateInit, jobList=jobListExd)

In [ ]:
num0List = []
num1List = []

# projector |1><1| of qubit 1
prj1 = tensor(np.identity(3), basis(level, 1) @ dagger(basis(level, 1)), np.identity(3))

# append the result to the list
for res0, res1 in zip(resultGrd, resultExd):
    psi0, psi1 = res0['state'], res1['state']
    rho0, rho1 = psi0 @ dagger(psi0), psi1 @ dagger(psi1)
    num0List.append(expect(prj1, rho0))
    num1List.append(expect(prj1, rho1))


# plot the result
plt.figure(figsize=(8, 6))
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 1')
plt.xlabel
plt.title('Ramsey on Q1')
plt.legend()
plt.show()

In [ ]:
# Fit the curve
para1Fit, _ = curve_fit(fit, tgList, num0List, [2 * pi * freq / 1.2, 0.])
para2Fit, _ = curve_fit(fit, tgList, num1List, [2 * pi * freq / 1.2, 0.])
step = 0.01
y1Fit = [fit(x, para1Fit[0], para1Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]
y2Fit = [fit(x, para2Fit[0], para2Fit[1]) for x in np.arange(tgList[0], tgList[-1], step)]

# Plot the curve
plt.figure(figsize=(8, 6))
plt.plot(np.arange(tgList[0], tgList[-1], step), y1Fit)
plt.plot(np.arange(tgList[0], tgList[-1], step), y2Fit)
plt.plot(tgList, num0List, '.b', label=r'$|00\rangle$')
plt.plot(tgList, num1List, '.r', label=r'$|01\rangle$')
plt.xlabel('Delayed time (ns)')
plt.ylabel('Population of excited state of qubit 1')
plt.xlabel
plt.title('Ramsey on Q1')
plt.legend()
plt.show()

In [ ]:
# Calculate the crosstalk strength
xiEst = abs(para1Fit[0] - para2Fit[0]) 
print(f'Coupling strength: {g12 * 1e3 / (2 * pi)} MHz')
print(f'ZZ crosstalk strength: {xiEst * 1e3 / (2 * pi)} MHz')

Due to the weaker coupling strength, the relatively small qubit frequency shift of $q_1$ indicates the weak ZZ crosstalk between qubit $q_1$ and $q_2$.

## Summary

After reading this tutorial on multi-qubit noisy simulator, users are encouraged to explore other advanced research which is different from this tutorial.

## References
\[1\] [Malekakhlagh, Moein, Easwar Magesan, and David C. McKay. "First-principles analysis of cross-resonance gate operation." Physical Review A 102.4 (2020): 042605.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.102.042605)

\[2\] [Magesan, Easwar, and Jay M. Gambetta. "Effective Hamiltonian models of the cross-resonance gate." Physical Review A 101.5 (2020): 052308.](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.101.052308)

\[3\] [Ku, Jaseung, et al. "Suppression of Unwanted Z Z Interactions in a Hybrid Two-Qubit System." Physical review letters 125.20 (2020): 200504.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.200504)